# Introduction

In this notebook we present a set of basic tests of the implementations
of null models provided by `pathcensus` package. All the null models
are also tested against an automated suite of unit test, but we additionally
provide the below examples as the notebook format is argurably much easier
to follow. We use  `igraph` package to generate graphs.

We defined all models following the formulas and terminology introduced in:

> [1] Squartini, T., Mastrandrea, R., & Garlaschelli, D. (2015). 
> Unbiased sampling of network ensembles. 
> New Journal of Physics, 17(2), 023052. https://doi.org/10.1088/1367-2630/17/2/023052

and:

> [2] Vallarano, N., Bruno, M., Marchese, E., Trapani, G., Saracco, F., Cimini, G., Zanon, M., & Squartini, T. (2021). Fast and scalable likelihood maximization for Exponential Random Graph Models with local constraints. Scientific Reports, 11(1), 15227. https://doi.org/10.1038/s41598-021-93830-4




In [1]:
import random
import numpy as np
import igraph as ig
from pathcensus.nullmodels import UBCM, UECM
from pathcensus.utils import rowsums, relclose

def add_random_weights(graph):
    graph = graph.copy()
    graph.es["weight"] = np.random.randint(1, 11, (graph.ecount(),))
    return graph

def make_er_graph(n, dbar):
    p = dbar / (n-1)
    return ig.Graph.Erdos_Renyi(n, p=p, directed=False)

def make_rgg(n, dbar):
    radius = np.sqrt(dbar/(np.pi*(n-1)))
    return ig.Graph.GRG(n, radius=radius, torus=True)

# Global parameters
# -----------------
N_NODES   = 100     # number of nodes in random graphs
KBAR      = 10      # expected average degree in random graphs
RTOL      = 1e-1    # relative tolerance when comparing simulated and expected values
N_SAMPLES = 1000    # number of samples using for stochastic testing of expectations

## Undirected Binary Configuration Model (UBCM)

This is a soft (canonical) configuration model for undirected, unweighted
networks. It is defined in Sec. 3.1 and Eq. (8) in [1].

For this model we will test whether node degrees are indeed reproduced
in expectation, which is exactly what the model should do. We will test
this on two small random graph with very different structure:

1. Erdős–Rényi random graph
2. Random geometric graph

Both graphs will have $100$ nodes and average degrees equal to $10$ approximately.

### ER random graph

In [2]:
random.seed(303)

graph  = make_er_graph(N_NODES, KBAR)
degseq = np.array(graph.degree())

ubcm = UBCM(graph)
ubcm.fit()

In [3]:
## TEST ANALYTICAL EXPECTED DEGREES
relclose(ubcm.ED, degseq, rtol=RTOL)

True

In [4]:
## TEST EXPECTATION THROUGH SAMPLING
expected = np.zeros_like(degseq, dtype=float)

for randomized in ubcm.sample(N_SAMPLES):
    # Sample graph realizations are adjacency matrices
    expected += rowsums(randomized)

expected = expected / N_SAMPLES

relclose(expected, degseq, rtol=RTOL) 

True

### Random geometric graph

In [5]:
random.seed(304)

graph  = make_rgg(N_NODES, KBAR)
degseq = np.array(graph.degree())

ubcm = UBCM(graph)
ubcm.fit()

In [6]:
## TEST ANALYTICAL EXPECTED DEGREES
relclose(ubcm.ED, degseq, rtol=RTOL)

True

In [7]:
## TEST EXPECTATION THROUGH SAMPLING
expected = np.zeros_like(degseq, dtype=float)

for randomized in ubcm.sample(N_SAMPLES):
    # Sample graph realizations are adjacency matrices
    expected += rowsums(randomized)

expected = expected / N_SAMPLES

relclose(expected, degseq, rtol=RTOL) 

True

## Undirected Enhanced Configuration Model

This null model constrains both expected degree sequence and strength
sequence. We test it again against ER and RGG networks, but this time
we also add random edge weights between $1$ and $10$.

### ER random graph

In [8]:
random.seed(305)

graph = make_er_graph(N_NODES, KBAR)
graph = add_random_weights(graph)
D     = np.array(graph.degree())
S     = np.array(graph.strength(weights="weight"))

uecm = UECM(graph)
uecm.fit()

In [9]:
## TEST ANALYTICAL EXPECTED DEGREES
relclose(uecm.ED, D, rtol=RTOL)

True

In [10]:
## TEST EXPECTATION THROUGH SAMPLING
expected = np.zeros_like(degseq, dtype=float)

for randomized in uecm.sample(N_SAMPLES):
    # Sample graph realizations are adjacency matrices
    randomized.data[:] = 1
    expected += rowsums(randomized)

expected = expected / N_SAMPLES

relclose(expected, D, rtol=RTOL) 

True

In [11]:
## TEST ANALYTICAL EXPECTED STRENGTHS
relclose(uecm.ES, S, rtol=RTOL)

True

In [12]:
## TEST EXPECTATION THROUGH SAMPLING
expected = np.zeros_like(degseq, dtype=float)

for randomized in uecm.sample(N_SAMPLES):
    # Sample graph realizations are adjacency matrices
    expected += rowsums(randomized)

expected = expected / N_SAMPLES

relclose(expected, S, rtol=RTOL) 

True

### Random geometric graph

In [13]:
random.seed(306)

graph = make_rgg(N_NODES, KBAR)
graph = add_random_weights(graph)
D     = np.array(graph.degree())
S     = np.array(graph.strength(weights="weight"))

uecm = UECM(graph)
uecm.fit()

In [14]:
## TEST ANALYTICAL EXPECTED DEGREES
relclose(uecm.ED, D, rtol=RTOL)

True

In [15]:
## TEST EXPECTATION THROUGH SAMPLING
expected = np.zeros_like(degseq, dtype=float)

for randomized in uecm.sample(N_SAMPLES):
    # Sample graph realizations are adjacency matrices
    randomized.data[:] = 1
    expected += rowsums(randomized)

expected = expected / N_SAMPLES

relclose(expected, D, rtol=RTOL) 

True

In [16]:
## TEST ANALYTICAL EXPECTED STRENGTHS
relclose(uecm.ES, S, rtol=RTOL)

True

In [17]:
## TEST EXPECTATION THROUGH SAMPLING
expected = np.zeros_like(degseq, dtype=float)

for randomized in uecm.sample(N_SAMPLES):
    # Sample graph realizations are adjacency matrices
    expected += rowsums(randomized)

expected = expected / N_SAMPLES

relclose(expected, S, rtol=RTOL) 

True